# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())
['establishments']

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query1 = {'scores.Hygiene': 20}
result1 = establishments.find(query1)

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query1))

# Display the first document in the results using pprint
pprint(result1[0])

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName' : {'$regex' : 'London'}, 'RatingValue' : {'$gte' : 4}}
result2 = establishments.find(query2)

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query2))

# Display the first document in the results using pprint
pprint(result2[0])

In [ ]:
# Convert the result to a Pandas DataFrame
result1_df = pd.DataFrame(result1)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result1_df))

# Display the first 10 rows of the DataFrame
result1_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query3 = {
    'RatingValue': 5, 
    'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
    'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)}
}

sort3 = [('scores.Hygiene', 1)]
limit3 = 5

result3 = list(establishments.find(query3).sort(sort3).limit(limit3))

# Print the results
pprint(result3)

In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match' : {'scores.Hygiene' : 0}}
group_query = {'$group' : {'_id' : '$LocalAuthorityName', 'count':{'$sum':1}}}
sort_values = {'$sort' : {'count' : -1}}

pipeline = [match_query, group_query, sort_values]

result4 = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print(len(result4))

# Print the first 10 results
pprint(result4[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
zero_hygiene_df = pd.DataFrame(result4)
# Display the number of rows in the DataFrame
print(len(zero_hygiene_df))
# Display the first 10 rows of the DataFrame
zero_hygiene_df.head(10)